In [18]:
import numpy as np
import Amazon_Movie_Parser as amp
import Graph_Amazon_Movies as grp
# save numpy array as csv file
from numpy import savetxt
# load numpy array from csv file
from numpy import loadtxt
import networkx as nx

In [2]:
# Normalization test
tt = np.array([[2, 4, 6], [1, 5, 8], [20, 4, 6]])
#print(tt.mean(axis=1))
#tt_norm=tt/tt.mean(axis=1)
tt_sum = tt.sum(axis=1)
print(tt.sum(axis=1))
print(np.tile(tt_sum, (3, 1)).T)
# divide each element in the row with the sum of the row so the sum will be equal to 1
tt_norm=tt/np.tile(tt_sum, (3, 1)).T
print(tt_norm)

[12 14 30]
[[12 12 12]
 [14 14 14]
 [30 30 30]]
[[0.16666667 0.33333333 0.5       ]
 [0.07142857 0.35714286 0.57142857]
 [0.66666667 0.13333333 0.2       ]]


In [57]:
def Normalize_Array(arr_to_normalize, axis=0):
    if axis == 1:
        arr_sum_per_row = arr_to_normalize.sum(axis=1)
        arr_norm = arr_to_normalize/np.tile(arr_sum_per_row, (arr_sum_per_row.shape[0], 1)).T
    elif axis == 0:
        arr_sum_per_row = arr_to_normalize.sum(axis=0)
        arr_norm = arr_to_normalize/np.tile(arr_sum_per_row, (arr_sum_per_row.shape[1], 1)).T
    return arr_norm

In [4]:
#edge_index = df_movies.query("movieId == 'A141HP4LYPW' and userId == 'B003AI2VGA'").index
edge_index = df_movies.query("movieId == 'A141HP4LYPW' and userId == 'B003AI2VGAzzzzz'").index
if len(edge_index) != 0:
    df_movies.iloc[edge_index[0], 2]
else:
    print('no edge found')
#df_movies.loc[edge_index[0], 'score']

NameError: name 'df_movies' is not defined

In [ ]:
arr_sum_per_row = arr_to_normalize.sum(axis=1)
arr_norm = arr_to_normalize/np.tile(arr_sum_per_row, (arr_sum_per_row.shape[0], 1)).T

In [8]:
lst_test = [('A141HP4LYPW', 'B003AI2VGA', '3.0'),
     ('B000063W1R', 'A141HP4LYPW', '4.0'),
     ('6304286961', 'A141HP4LYPW', '5.0'),
     ('A141HP4LYPW', '5556167281', '5.0')]

# create a sample graph with edges
def create_sample_graph(lst_test):
    '''
    lst_test = [('A141HP4LYPW', 'B003AI2VGA', '3.0'),
     ('B000063W1R', 'A141HP4LYPW', '4.0'),
     ('6304286961', 'A141HP4LYPW', '5.0'),
     ('A141HP4LYPW', '5556167281', '5.0')]
    '''
    print(lst_test)

    df_movies = amp.Reorganize_Edges_DataFrame(lst_test)
    
    return df_movies

In [22]:
lst_test = [('A141HP4LYPW', 'B003AI2VGA', '3.0'),
     ('B000063W1R', 'A141HP4LYPW', '4.0'),
     ('6304286961', 'A141HP4LYPW', '5.0'),
     ('A141HP4LYPW', '5556167281', '5.0')]

grp1 = grp.Graph_Amazon()
grp_test=grp1.Create_Graph_From_List_WITH_Weight(lst_test)

In [62]:
grp_test.edges.data('weight', default=1)

EdgeDataView([('A141HP4LYPW', 'B003AI2VGA', '3.0'), ('A141HP4LYPW', 'B000063W1R', '4.0'), ('A141HP4LYPW', '6304286961', '5.0'), ('A141HP4LYPW', '5556167281', '5.0')])

In [60]:
grp_test.nodes

NodeView(('A141HP4LYPW', 'B003AI2VGA', 'B000063W1R', '6304286961', '5556167281'))

In [73]:
list(grp_test.nodes)[1]

'B003AI2VGA'

In [64]:
grp_test.has_edge('A141HP4LYPW', 'B003AI2VGAx')

False

In [70]:
gdata=grp_test.get_edge_data('A141HP4LYPW', 'B003AI2VGA')
print(gdata)

In [72]:
gdata['weight']

'3.0'

In [74]:
# this gives the first n nodes as movies and the rest as users
# it other words top_nodes (node group with fewer nodes in it) are listed first
P = nx.to_numpy_matrix(grp_test)  ## corresponds to M in the notes
P_norm = Normalize_Array(P)

n_nodes = P.shape[0]   ## P = nxn matrix
R = np.diag(np.ones(n_nodes))
R_zero = R.copy()
beta = 0.15
print(P_norm)
print(R)
print(R_zero)

[[0.         0.17647059 0.23529412 0.29411765 0.29411765]
 [1.         0.         0.         0.         0.        ]
 [1.         0.         0.         0.         0.        ]
 [1.         0.         0.         0.         0.        ]
 [1.         0.         0.         0.         0.        ]]
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [10]:
R_P = R.copy()
R_R = R.copy()

In [78]:
R_P = (1-beta)*np.dot(P_norm, R_P) + beta*R_zero
print(R_P)

# since there is no direction, using P or R as 
R_R = (1-beta)*np.dot(R_R, P_norm) + beta*R_zero
print(R_R)

[[0.33667594 0.03875625 0.051675   0.06459375 0.50829906]
 [0.24264844 0.21364906 0.08230656 0.0932875  0.36810844]
 [0.24264844 0.06364906 0.23230656 0.0932875  0.36810844]
 [0.24264844 0.06364906 0.08230656 0.2432875  0.36810844]
 [0.24264844 0.06364906 0.08230656 0.0932875  0.51810844]]
[[0.33667594 0.11705719 0.15607625 0.19509531 0.19509531]
 [0.21961875 0.26124375 0.148325   0.18540625 0.18540625]
 [0.21961875 0.11124375 0.298325   0.18540625 0.18540625]
 [0.21961875 0.11124375 0.148325   0.33540625 0.18540625]
 [0.29791969 0.09742594 0.12990125 0.16237656 0.31237656]]
